## ДЗ

1. Написать бота который будет проверять доступность сайта

In [ ]:
from telebot import *
import requests

url = ''
bot = telebot.TeleBot('...')

@bot.message_handler(commands=["start"])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Отправь мне ссылку на любой сайт и я проверю его доступность.')

@bot.message_handler(content_types=["text"])
def check_url(message):
    response = None
    global url
    if message.text.find('http') == -1:
        url = 'https://' + message.text
    else:
        url = message.text
    try:
        response = requests.get(url)
        keyboard = types.InlineKeyboardMarkup()
        key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes');
        keyboard.add(key_yes)
        key_no = types.InlineKeyboardButton(text='В другой раз', callback_data='no'); 
        keyboard.add(key_no)
        bot.send_message(message.chat.id, 'Сайт доступен, хочешь перейти на него?', reply_markup=keyboard)
    except: 
        if response is not None:
            bot.send_message(message.chat.id, 'Сайт в настоящее время недоступен и вернул HTTP ответ:', response.status_code)
        else:
            bot.send_message(message.chat.id, 'Проверьте ссылку')
            
@bot.callback_query_handler(func = lambda call: True)
def callback_worker(call):
    if call.data == "yes":
        keyboard = types.InlineKeyboardMarkup()
        button_url = types.InlineKeyboardButton(text=f'{url}', url=f'{url}')
        keyboard.add(button_url)
        bot.send_message(call.message.chat.id, 'Счастливого пути!', reply_markup=keyboard)
    elif call.data == "no":
        bot.send_message(call.message.chat.id, 'Ладно.')
        
bot.polling(none_stop=True, interval=0)

2. Написать бота который будет анализировать текст - (Статистика, количество уникальных слов, самые популярные слова, кроме союзов и предлогов, количество предложений)

In [ ]:
from telebot import *
import re
from collections import Counter

conj_prep = '''что, будто, если, когда, словно, чтобы, как, ибо, пока, и, тоже, также, ни, а, но, однако,
зато, же, только, или, либо, то, когда, пока, едва, лишь, вследствие, как, точно, будто, 
чтобы, дабы, если, когда, коли, ежели, раз, хотя, поэтому, потому'''
current_page = 1
pages = 1
words = []

keyboard = telebot.types.InlineKeyboardMarkup()
keyboard.row(   telebot.types.InlineKeyboardButton('<', callback_data = '<'),
                telebot.types.InlineKeyboardButton('' + str(current_page), callback_data = ' '),
                telebot.types.InlineKeyboardButton('>', callback_data = '>'))

bot = telebot.TeleBot('...')

@bot.message_handler(commands=["start"])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Пришли мне любой текст и я составлю по нему статистику.')
    
@bot.message_handler(content_types=["text"])
def analyzer(message):
    global words
    text = message.text
    words = list(filter(None, re.split(r'[\s!\,\.:\—\?\-\;\(\)]+', text))) 
    current_page = 1
    pages = 1
    
    update_stat(message, False)
    
    clean_words = filter(lambda w: w not in conj_prep, words)    
    bot.send_message(message.chat.id, 'Самое популярные слова:\n' + '\n'.join((c[0] + ' - ' + str(c[1]) + ' раз(а)') for c in Counter(clean_words).most_common(5)))
    bot.send_message(message.chat.id, 'Самое длинное:\n' + max(words, key = len))
    
    if text.strip()[-1] == '.':
        bot.send_message(message.chat.id, 'Количество предложений: ' + str(message.text.count('.')))
    else:
        bot.send_message(message.chat.id, 'Количество предложений: ' + str(message.text.count('.') + 1))


@bot.callback_query_handler(func = lambda call: True)
def turn_page(call):
    global current_page
    if call.data == '>' and current_page != pages:
        current_page += 1
        update_stat(call, True)
    elif call.data == '<' and current_page != 1:
        current_page -= 1
        update_stat(call, True)
           
    
def update_stat(message, isUpdate): 
    global current_page
    global pages
    global words
    
    pages = len(words) // 20 + 1    
    l = len(''.join(words))
    result = []
    
    keyboard = telebot.types.InlineKeyboardMarkup()
    keyboard.row(   telebot.types.InlineKeyboardButton('<', callback_data = '<'),
                    telebot.types.InlineKeyboardButton('' + str(current_page), callback_data = ' '),
                    telebot.types.InlineKeyboardButton('>', callback_data = '>'))
                    
    for w in words[(current_page-1)*20:current_page*20]:
        result.append(w + ' - {0:.2f}%\n'.format(len(w)/l*100))
    
    if isUpdate == False:
        bot.send_message(message.chat.id, ''.join(result), reply_markup = keyboard)
    else:
        bot.edit_message_text(chat_id = message.message.chat.id, message_id = message.message.message_id, text = ''.join(result), reply_markup = keyboard)    
    
bot.polling(none_stop=True, interval=0)

3. Бот калькулятор
*с кнопками

In [ ]:
from telebot import *
import re

bot = telebot.TeleBot('...')
value = ''
prev_value = ''

keyboard = telebot.types.InlineKeyboardMarkup()
keyboard.row(   telebot.types.InlineKeyboardButton('CE', callback_data = 'CE'),
                telebot.types.InlineKeyboardButton('C', callback_data = 'C'),
                telebot.types.InlineKeyboardButton('<=', callback_data = '<='),
                telebot.types.InlineKeyboardButton('/', callback_data = '/'))
                
keyboard.row(   telebot.types.InlineKeyboardButton('7', callback_data = '7'),
                telebot.types.InlineKeyboardButton('8', callback_data = '8'),
                telebot.types.InlineKeyboardButton('9', callback_data = '9'),
                telebot.types.InlineKeyboardButton('*', callback_data = '*'))
                
keyboard.row(   telebot.types.InlineKeyboardButton('4', callback_data = '4'),
                telebot.types.InlineKeyboardButton('5', callback_data = '5'),
                telebot.types.InlineKeyboardButton('6', callback_data = '6'),
                telebot.types.InlineKeyboardButton('-', callback_data = '-'))
                
keyboard.row(   telebot.types.InlineKeyboardButton('1', callback_data = '1'),
                telebot.types.InlineKeyboardButton('2', callback_data = '2'),
                telebot.types.InlineKeyboardButton('3', callback_data = '3'),
                telebot.types.InlineKeyboardButton('+', callback_data = '+'))
                
keyboard.row(   telebot.types.InlineKeyboardButton('+-', callback_data = '+-'),
                telebot.types.InlineKeyboardButton('0', callback_data = '0'),
                telebot.types.InlineKeyboardButton('.', callback_data = '.'),
                telebot.types.InlineKeyboardButton('=', callback_data = '='))

@bot.message_handler(commands=["start"])
def start(message):
    global value
    if value == '':
        bot.send_message(message.chat.id, '0', reply_markup = keyboard)
    else:
        bot.send_message(message.chat.id, value, reply_markup = keyboard)
    
@bot.callback_query_handler(func = lambda call: True)
def callback_worker(call):
    global value, prev_value
    if call.data == 'CE':
        value = ''.join(re.split(r'[\*\/\+\-]', value)[:-1])
    elif call.data == 'C':
        value = ''
    elif call.data == '<=':
        if value != '':
            value = value[:-1]
    elif call.data == '+-':
        if value[0] == '-':
            value = value[1:]
        else:
            value = '-' + value
    elif call.data == '=':
        try:
            value = str(eval(value))
        except:
            value = 'Error'
    else:
        value += call.data
        
    if value != prev_value:
        if value == '':
            bot.edit_message_text(chat_id = call.message.chat.id, message_id = call.message.message_id, text = '0', reply_markup = keyboard)
            prev_value = '0'
        else:
            bot.edit_message_text(chat_id = call.message.chat.id, message_id = call.message.message_id, text = value, reply_markup = keyboard)
            prev_value = value
   
    if value == 'Error':
        value = ''


bot.polling(none_stop=True, interval=0)